In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/midcp_wave.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.tail()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime
5016,2024-12-05T09:15:00+05:30,13019.1000,13021.0500,12912.7000,12921.1000,12538.1831,33.6051,0,33.6051,2024-12-05 09:15:00+05:30
5017,2024-12-05T10:15:00+05:30,12920.8500,12942.4500,12883.2000,12884.5500,12548.1900,24.1513,0,24.1513,2024-12-05 10:15:00+05:30
5018,2024-12-05T11:15:00+05:30,12884.5000,12927.3000,12868.5000,12921.2500,12558.5506,23.0916,0,23.0916,2024-12-05 11:15:00+05:30
5019,2024-12-05T12:15:00+05:30,12921.6500,12937.5500,12912.2500,12929.6000,12566.6144,22.3792,0,22.3792,2024-12-05 12:15:00+05:30
5020,2024-12-05T13:15:00+05:30,12930.3500,12937.3500,12914.7500,12926.4000,12574.5525,20.1495,0,20.1495,2024-12-05 13:15:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2022, 1, 10),
 datetime.date(2022, 1, 11),
 datetime.date(2022, 1, 12),
 datetime.date(2022, 1, 13),
 datetime.date(2022, 1, 14),
 datetime.date(2022, 1, 17),
 datetime.date(2022, 1, 18),
 datetime.date(2022, 1, 19),
 datetime.date(2022, 1, 20),
 datetime.date(2022, 1, 21),
 datetime.date(2022, 1, 24),
 datetime.date(2022, 1, 25),
 datetime.date(2022, 1, 27),
 datetime.date(2022, 1, 28),
 datetime.date(2022, 1, 31),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 2, 2),
 datetime.date(2022, 2, 3),
 datetime.date(2022, 2, 4),
 datetime.date(2022, 2, 7),
 datetime.date(2022, 2, 8),
 datetime.date(2022, 2, 9),
 datetime.date(2022, 2, 10),
 datetime.date(2022, 2, 11),
 datetime.date(2022, 2, 14),
 datetime.date(2022, 2, 15),
 datetime.date(2022, 2, 16),
 datetime.date(2022, 2, 17),
 datetime.date(2022, 2, 18),
 datetime.date(2022, 2, 21),
 datetime.date(2022, 2, 22),
 datetime.date(2022, 2, 23),
 datetime.date(2022, 2, 24),
 datetime.date(2022, 2, 25),
 datetime.date(2022, 

In [9]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [37]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0005
LEVERAGE_ = 5

In [38]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    df['EMA_Trend'] = df['EMA_20'] > df['EMA_20'].shift(1)
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['close'] > df['X_High'].shift(1) * (1 + (pct / 100))) & 
        (df['EMA_Trend']) & 
        (df['close'] > df['EMA_20'])
    )
    
    return df

In [39]:
# def backtest(df):
#     position = 0
#     entry_price = 0
#     trades = []
#     trailing_stop = None
    
#     for i in range(len(df)):
#         if df.loc[i, 'Buy_Signal'] and position == 0:
#             position = 1
#             # entry_price = df.loc[i, 'close']
#             trailing_stop_long = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Buy', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_long = df.loc[i, 'datetime']
#             entry_price_long = df.loc[i, 'close']
#             initial_sl_long = trailing_stop_long
        
#         elif df.loc[i, 'Sell_Signal'] and position == 0:
#             position = -1
#             entry_price = df.loc[i, 'close']
#             trailing_stop_short = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Sell', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_short = df.loc[i, 'datetime']
#             entry_price_short = df.loc[i, 'close']
#             initial_sl_short = trailing_stop_short
        
#         elif position == 1:
#             if df.loc[i, 'low'] < trailing_stop_long:
#                 # trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#                 exit_time_long = df.loc[i, 'datetime']
#                 exit_price_long = trailing_stop_long
#                 points_long = exit_price_long - entry_price_long
                
#             elif df.loc[i, 'Add_Long']:
#                 # trades.append({'Type': 'Add Long', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 entry_time_long = df.loc[i, 'datetime']
#                 entry_price_long = df.loc[i, 'close']
#                 initial_sl_long = trailing_stop_long
#                 trailing_stop_long = max(trailing_stop_long, df.loc[i, 'EMA_20'])
        
#         elif position == -1:
#             if df.loc[i, 'high'] > trailing_stop:
#                 trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#             elif df.loc[i, 'Add_Short']:
#                 trades.append({'Type': 'Add Short', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 trailing_stop = min(trailing_stop, df.loc[i, 'EMA_20'])
    
#     return pd.DataFrame(trades)

In [40]:
def backtest(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_long = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = df.loc[i, 'X_Low']

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_High'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to long position
        if long_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , long_initial_sl , 'Initial SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0
            
            elif df.loc[i, 'close'] <= long_trailing_stop:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_long, tradebook], ignore_index=True)
    return tradebook


In [41]:
# def backtest(df, y_days):
#     # Variables for long trades
#     long_position = 0  # 0 = no position, 1 = long
#     long_entry_price = 0
#     long_entry_date = None
#     long_trades = []
#     tradebook = pd.DataFrame()
#     tradebook_long = pd.DataFrame()
#     tradebook_short = pd.DataFrame()
#     long_trailing_stop = None
#     # Variables for short trades
#     short_position = 0  # 0 = no position, -1 = short
#     short_entry_price = 0
#     short_entry_date = None
#     short_trades = []
#     short_trailing_stop = None

#     can_add_long = False
#     can_add_short = False

#     trade_number = 0

#     for i in range(len(df)):
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']
        
#         if df.loc[i, 'Buy_Signal'] and long_position == 0:
#             long_position = 1
#             long_entry_price = df.loc[i, 'close']
#             long_entry_date = df.loc[i, 'datetime']
#             long_trailing_stop = df.loc[i, 'EMA_20']
#             long_initial_sl = df.loc[i, 'X_Low']

#             tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Long',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': long_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         # Exit or add to long position
#         if long_position == 1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'low'] <= long_initial_sl:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , long_initial_sl]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] <= long_trailing_stop:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1

#             elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
#                 # Previous Y Candle Low Breached, confirmation found. Add position on high break
#                 can_add_long = True

#             if can_add_long and long_position:
#                 if df.loc[i, 'high'] >= y_days_high:
#                     tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Long',
#                         'Entry_Price': y_days_high,
#                         'Initial SL': long_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_long = False

#     for i in range(len(df)):
        
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']

#         # Entry signals for short trades
#         if df.loc[i, 'Sell_Signal'] and short_position == 0:
#             short_position = -1
#             short_entry_price = df.loc[i, 'close']
#             short_entry_date = df.loc[i, 'datetime']
#             short_trailing_stop = df.loc[i, 'EMA_20']
#             short_initial_sl = df.loc[i, 'X_High']

#             tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Short',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': short_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         if short_position == -1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'high'] >= short_initial_sl:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] ,short_initial_sl]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] >= short_trailing_stop:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1

#             elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
#                 # Previous Y Candle High Breached, confirmation found. Add position on low break
#                 can_add_short = True

#             if can_add_short and short_position:
#                 if df.loc[i, 'low'] <= y_days_low:
#                     tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Short',
#                         'Entry_Price': y_days_low,
#                         'Initial SL': short_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_short = False
    
#     tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
#     return tradebook


In [42]:
ema_length = 55
x_days = 9
# y_days = 4
pct = 0.5
RPT = 3
df = bnf_1min
df = calculate_signals(df, ema_length,x_days, pct)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_34943/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


In [43]:
import numpy as np
portfolio = 10000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio * 8 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [44]:
# tb['Cumulative_PnL'] = tb['PnL w cs'].cumsum()

In [45]:
# tb.to_csv('GOLD TRADE CHECK JJPP.csv', index=False)

In [46]:
tb = tb.reset_index()
# tb

In [47]:
import matplotlib.pyplot as plt

def plot_cumulative_pnl(tb, title="Cumulative PnL Over Time"):
    if 'Cumulative_PnL' not in tb.columns:
        raise ValueError("The DataFrame must contain a 'Cumulative_PnL' column.")

    # Plot cumulative PnL
    plt.figure(figsize=(12, 3))
    plt.plot(tb.index, tb['Cumulative_PnL'], label='Cumulative PnL', color='blue', linewidth=2)
    plt.title(title, fontsize=8)
    plt.xlabel('Trade Index', fontsize=6)
    plt.ylabel('Cumulative PnL', fontsize=6)
    plt.legend(fontsize=6)
    plt.grid(alpha=0.5)
    plt.show()


In [48]:
# tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
# tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()

In [49]:
# tb_long_only = tb_long_only[tb_long_only['Trade Year'] >= 2017]

In [50]:
# plot_cumulative_pnl(tb_long_only)
# print(tb_long_only['PnL w cs'].sum())

In [51]:
# plot_cumulative_pnl(tb_short_only)
# print(tb_short_only['PnL w cs'].sum())

In [52]:
# plot_cumulative_pnl(tb)
# print(tb['PnL w cs'].sum())

In [53]:
# plot_cumulative_pnl(tb2)
# print(tb2['PnL w cs'].sum())

In [54]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2022, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2022, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [55]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [56]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2022,30.5780,29,37.9310,5.7128,-1.7924,-9.0872,3.3650,...
2023,87.6314,22,50.0000,10.3334,-2.3669,-10.9327,8.0156,...
2024,1.3724,33,27.2727,4.8422,-1.8351,-15.7892,0.0869,...
Overall,119.5818,84,36.9048,7.0996,-1.9328,-21.8796,5.4655,...


# Gold 4H X_Days High, Y_Days Low 

In [57]:
tb1 = tb[tb['Trade_Type'] == 'Long']
tb2 = tb[tb['Trade_Type'] == 'Add_Long']
tb1['ROI%'].sum() , tb2['ROI%'].sum()

(119.5818478051755, 0)

In [58]:
tb.tail(25)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
59,59,59,2024-03-07 09:15:00+05:30,2024-03-07 14:15:00+05:30,Long,10921.3500,10866.8333,10799.9222,10892.8000,Trailing SL Hit,-28.5500,10.9071,-39.4571,2470.6044,-97482.8224,-0.9748,2024
60,60,60,2024-03-21 14:15:00+05:30,2024-03-22 09:15:00+05:30,Long,10434.1500,10275.2889,10256.1167,10394.1500,Trailing SL Hit,-40.0000,10.4141,-50.4141,1685.0777,-84951.7600,-0.8495,2024
61,61,61,2024-03-26 11:15:00+05:30,2024-04-15 09:15:00+05:30,Long,10480.5500,10421.9056,10396.3444,10812.2000,Trailing SL Hit,331.6500,10.6464,321.0036,3562.7103,1143642.9208,11.4364,2024
62,62,62,2024-04-18 12:15:00+05:30,2024-04-18 14:15:00+05:30,Long,10896.5000,10837.3667,10795.7389,10795.7389,Initial SL Hit,-100.7611,10.8461,-111.6072,2977.3391,-332292.5759,-3.3229,2024
63,63,63,2024-04-23 09:15:00+05:30,2024-04-23 10:15:00+05:30,Long,10781.7000,10719.6111,10695.7500,10763.8500,Trailing SL Hit,-17.8500,10.7728,-28.6228,3490.4014,-99904.9738,-0.9990,2024
64,64,64,2024-04-26 09:15:00+05:30,2024-05-07 10:15:00+05:30,Long,10953.9000,10871.0111,10847.3333,10906.1000,Trailing SL Hit,-47.8000,10.9300,-58.7300,2815.1392,-165333.1248,-1.6533,2024
65,65,65,2024-05-13 12:15:00+05:30,2024-05-30 09:15:00+05:30,Long,10953.4500,10866.8111,10806.4222,11404.7500,Trailing SL Hit,451.3000,11.1791,440.1209,2040.4308,898036.2214,8.9804,2024
66,66,66,2024-06-03 09:15:00+05:30,2024-06-04 09:15:00+05:30,Long,11714.4500,11369.7278,11346.4833,11346.4833,Initial SL Hit,-367.9667,11.5305,-379.4971,815.2912,-309400.6885,-3.0940,2024
67,67,67,2024-06-05 13:15:00+05:30,2024-07-02 12:15:00+05:30,Long,11407.3500,11094.7167,10920.5056,12171.3500,Trailing SL Hit,764.0000,11.7894,752.2106,616.2133,463522.1734,4.6352,2024
68,68,68,2024-07-04 09:15:00+05:30,2024-07-10 09:15:00+05:30,Long,12389.0500,12291.8833,12266.6667,12284.4500,Trailing SL Hit,-104.6000,12.3368,-116.9367,2451.3142,-286648.7131,-2.8665,2024


In [59]:
# tb.to_csv('SENSEX_jjpp_55_9_HALF_PCT_0_PYR.csv')

In [60]:
# tb_long_only.to_csv('GOLD Range JJPP.csv', index=False)

In [61]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.25, 0.5, 0.75, 1]
RPT_ = 3
stats_dictionary = {}

for ema_length in range(20, 81, 2):
    for x_days in range(3, 37, 3):
            for pct in pct_range:
                # RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, PCT:{pct}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest(df)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                    
                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 10:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                            stats_dictionary[x] = y
                        break

EMA:20, X:3, PCT:0.25


/tmp/ipykernel_34943/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:20, X:3, PCT:0.5


/tmp/ipykernel_34943/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:20, X:3, PCT:0.75


...

/tmp/ipykernel_34943/2639050160.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:0.75


/tmp/ipykernel_34943/2639050160.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:1


/tmp/ipykernel_34943/2639050160.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


# MIDCP 1Hr

In [62]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
for x, y in sorted_stats.items():
    print(y.to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2022      28.7260           24  37.5000                6.0334             -1.7050      -5.3145       5.4052  EMA:34, X:3, PCT:0.75
2023      67.6400           15  53.3333               10.5532             -2.3979      -4.5208      14.9621  EMA:34, X:3, PCT:0.75
2024      11.2124           20  35.0000                5.3514             -2.1873      -6.4123       1.7486  EMA:34, X:3, PCT:0.75
Overall  107.5784           59  40.6780                7.3411             -2.0179      -7.1608      15.0233  EMA:34, X:3, PCT:0.75
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2022      27.5440           25  36.0000                6.1138             -1.7175      -5.3145       5.1828  EMA:32, X:3, PCT:0.75
2023      67.8882           15  53.3333               10.5842             -2.3979  